In [1]:
import argparse
import datetime
import hashlib
import os
import os.path
import shutil
import smtplib
import subprocess
import sys
import time
import urllib.request
from contextlib import suppress
from datetime import datetime
from io import StringIO

import natsort
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys


def ishtml(filename):
    if not "." in filename:
        return False
    exts = ["html"]
    ext = filename.split(".")[-1]
    if ext in exts:
        return True
    return False

def listhtmlonly(dir):
    return natsort.natsorted([name for name in next(os.walk(os.path.join(dir, '.')))[2] if ishtml(name)])

In [2]:
options = webdriver.ChromeOptions()
#options.add_argument("user-data-dir=C:\\Users\\user\\AppData\\Local\\chrome\\User Data")
#options.binary_location = "C:\\Program Files (x86)\\google\\Application\\chrome.exe"
driver = webdriver.Chrome(options=options)

In [3]:
driver.get("https://goodinfo.tw/")

In [4]:
# 取得所有四位數的一般股票代碼
option_selenium = driver.find_elements("css selector", "#dlSTOCK_ID_NM > option")
option_value = [i.get_attribute('value') for i in option_selenium]
code = [i.split(" ")[0] for i in option_value]
filtered_code = [i for i in code if len(i) == 4]
len(filtered_code)

2098

In [7]:
# 已經下載完成的股票代碼
html_filename = listhtmlonly("")
finished = [i.split(".")[0] for i in html_filename]
len(finished)

2094

In [8]:
# 爬蟲前記得先更改瀏覽器下載位置和是否跳出下載確認對話框

unfinished = list(set(filtered_code) - set(finished))
unfinished.sort()
unfinished.remove('0000')
unfinished.remove('0001')

for i in unfinished:
    try:
        driver.get("https://goodinfo.tw/tw/StockDividendPolicy.asp?STOCK_ID={}".format(i))

        # 按下匯出HTML按鈕的js
        js = "export2html(divDetail.innerHTML,'{}.html');".format(i)
        driver.execute_script(js)
    except:
        print(i + ' Fail')
    
    # 間隔不能太短 一定時間內又不能太多 需要多爬幾次
    time.sleep(4)

6917 Fail
6937 Fail


In [9]:
html_filename = listhtmlonly("")
dfs = [pd.read_html(i)[0] for i in html_filename]
dfs[0]

股 利 政 策                                                      ...  \
   股利 發放 年度 股東股利 (元/股)                             股利總計          ...   
   股利 發放 年度       現金股利          股票股利       股利 合計 現金 (億) 股票 (千張)  ...   
   股利 發放 年度         盈餘 公積    合計   盈餘 公積 合計 股利 合計 現金 (億) 股票 (千張)  ...   
0      2023          -  -     -    -  -  -     -      -       -  ...   
1      2022          5  0     5    0  0  0     5      0       0  ...   
2         ∟        1.8  0   1.8    0  0  0   1.8      0       0  ...   
3         ∟        3.2  0   3.2    0  0  0   3.2      0       0  ...   
4      2021        3.4  0   3.4    0  0  0   3.4      0       0  ...   
5         ∟       0.35  0  0.35    0  0  0  0.35      0       0  ...   
6         ∟       3.05  0  3.05    0  0  0  3.05      0       0  ...   
7      2020        3.6  0   3.6    0  0  0   3.6      0       0  ...   
8         ∟        0.7  0   0.7    0  0  0   0.7      0       0  ...   
9         ∟        2.9  0   2.9    0  0  0   2.9      0       0  ...   
10     2019          3  0     3    0  0  0     3      0       0  ...   
11        ∟        0.7  0   0.7    0  0  0   0.7      0       0  ...   
12        ∟        2.3  0   2.3    0  0  0   2.3      0       0  ...   
13     2018        2.9  0   2.9    0  0  0   2.9      0       0  ...   
14        ∟        0.7  0   0.7    0  0  0   0.7      0       0  ...   
15        ∟        2.2  0   2.2    0  0  0   2.2      0       0  ...   
16     2017        2.4  0   2.4    0  0  0   2.4      0       0  ...   
17        ∟        0.7  0   0.7    0  0  0   0.7      0       0  ...   
18        ∟        1.7  0   1.7    0  0  0   1.7      0       0  ...   
19     2016       0.85  0  0.85    0  0  0  0.85      0       0  ...   
20     2015          2  0     2    0  0  0     2      0       0  ...   
21     2014       1.55  0  1.55    0  0  0  1.55      0       0  ...   
22     2013       1.35  0  1.35    0  0  0  1.35      0       0  ...   
23     2012       1.85  0  1.85    0  0  0  1.85      0       0  ...   
24     2011       1.95  0  1.95    0  0  0  1.95      0       0  ...   
25     2010        2.2  0   2.2    0  0  0   2.2      0       0  ...   
26     2009          1  0     1    0  0  0     1      0       0  ...   
27     2008          2  0     2    0  0  0     2      0       0  ...   
28     2007        2.5  0   2.5    0  0  0   2.5      0       0  ...   
29     2006          4  0     4    0  0  0     4      0       0  ...   
30     2005       1.85  0  1.85    0  0  0  1.85      0       0  ...   
31     2004          0  0     0    0  0  0     0      0       0  ...   
32     2003          -  -     -    -  -  -     -      -       -  ...   
33       累計       43.4  0  43.4    0  0  0  43.4    NaN     NaN  ...   

   殖 利 率 統 計                            盈餘分配率統計                             
   殖 利 率 統 計                            盈餘分配率統計                             
     股價統計(元)       年均殖利率(%)            股利 所屬 期間 EPS (元) 盈餘分配率(%)            
          最低    年均       現金   股票    合計 股利 所屬 期間 EPS (元)       配息   配股   合計  
0     108.45   113        -    -     -        -       -        -    -    -  
1       96.5   122     4.09    0  4.09     2022       -        -    -    -  
2          -     -     1.47    0  1.47     22H2       -        -    -    -  
3          -     -     2.62    0  2.62     22H1       -        -    -    -  
4      121.2   137     2.48    0  2.48     2021       -        -    -    -  
5          -     -     0.26    0  0.26     21H2       -        -    -    -  
6          -     -     2.23    0  2.23     21H1       -        -    -    -  
7      67.25  96.9     3.71    0  3.71     2020       -        -    -    -  
8          -     -     0.72    0  0.72     20H2       -        -    -    -  
9          -     -     2.99    0  2.99     20H1       -        -    -    -  
10        72    83     3.61    0  3.61     2019       -        -    -    -  
11         -     -     0.84    0  0.84     19H2       -        -    -    -  
12         -     -     2.77    0  2

In [10]:
dfs[0].iloc[:, [0,18]]

,股 利 政 策,殖 利 率 統 計
,股利 發放 年度,殖 利 率 統 計
,股利 發放 年度,年均殖利率(%)
,股利 發放 年度,合計
0,2023,-
1,2022,4.09
2,∟,1.47
3,∟,2.62
4,2021,2.48
5,∟,0.26
6,∟,2.23


In [11]:
df = dfs[0].iloc[:, [0,18]]
df.columns = ['年度', '殖利率']
df[df.年度 != '∟']

,年度,殖利率
0,2023,-
1,2022,4.09
4,2021,2.48
7,2020,3.71
10,2019,3.61
13,2018,3.55
16,2017,3.05
19,2016,1.28
20,2015,3.01
21,2014,2.43


In [21]:
for i in html_filename:
    df_tmp = pd.read_html(i)[0]
    df_tmp = df_tmp.iloc[1:, [0,18]]
    df_tmp.columns = ['年度', i.split(".")[0]]
    df_tmp = df_tmp[df_tmp.年度 != '∟']
    df_tmp = df_tmp[df_tmp.年度 != '累計']
    df = df_tmp.copy()
    break
for i in html_filename[1:]:
    df_tmp = pd.read_html(i)[0]
    df_tmp = df_tmp.iloc[1:, [0,18]]
    df_tmp.columns = ['年度', i.split(".")[0]]
    df_tmp = df_tmp[df_tmp.年度 != '∟']
    df_tmp = df_tmp[df_tmp.年度 != '累計']
    df = df.merge(df_tmp, on='年度', how='outer')
df

,年度,0050,0051,0052,0053,0055,0056,0057,0061,1101,...,9945,9946,9949,9950,9951,9955,9957,9958,9960,9962
0,2022,4.09,4.35,9.63,2.91,3.7,7.17,0,0,4.93,...,11.1,5.24,0,0,4.89,0,0,3.02,6.57,6.69
1,2021,2.48,3.22,0.82,2.56,3.48,5.42,0,0,7,...,8.83,5.14,0,0,5.31,0,0,2.64,3.53,0
2,2020,3.71,2.97,2.4,2.88,3.68,5.61,0,0,7.05,...,13.8,9.4,0,0,4.5,0,0,1.01,2.81,2.22
3,2019,3.61,3.68,1.83,4.18,3.88,6.7,0,0,9.77,...,4.52,4.87,0,0,4.68,0,0,0.43,1.46,8.15
4,2018,3.55,3.13,3.24,3.14,3.75,5.62,0,0,6.37,...,5.37,10,0,0,4.74,0,0,0.46,0,5.74
5,2017,3.05,2.89,2.88,3.34,3.23,3.78,0,0,4.13,...,5.91,4.34,0,2.14,3.27,0,0,2.62,1.23,5.67
6,2016,1.28,2.78,5.99,3.53,3.42,5.67,4.22,0,4.03,...,0,7.86,0,4.63,3.08,0,0,4,1.86,0
7,2015,3.01,3.06,0,3.22,3.01,4.33,0,0,6.58,...,9.04,2.2,1.92,5.15,3.94,0,0,3.12,3.82,2.15
8,2014,2.43,2.5,6.38,3.24,2.8,4.12,0,0,5.03,...,8.47,10.6,3.1,3.7,4.03,0,0,6.44,5.04,5.63
9,2013,2.4,2.85,0,3.16,2.7,3.61,0,0,4.77,...,4.83,7.95,3.63,3.99,2.74,1.31,0,5.14,6.28,8.86


In [41]:
df2 = df.T.rename({'年度': '股票代號╲年度'}, axis='rows')
df2

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
股票代號╲年度,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,...,1979,1978,1977,1976,1975,1974,1973,1972,1971,1970
0050,4.09,2.48,3.71,3.61,3.55,3.05,1.28,3.01,2.43,2.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0051,4.35,3.22,2.97,3.68,3.13,2.89,2.78,3.06,2.5,2.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0052,9.63,0.82,2.4,1.83,3.24,2.88,5.99,0,6.38,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0053,2.91,2.56,2.88,4.18,3.14,3.34,3.53,3.22,3.24,3.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9955,0,0,0,0,0,0,0,0,0,1.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9957,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9958,3.02,2.64,1.01,0.43,0.46,2.62,4,3.12,6.44,5.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9960,6.57,3.53,2.81,1.46,0,1.23,1.86,3.82,5.04,6.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df2.to_csv('test.csv', encoding='cp950', header=False, index=True)

In [46]:
pd.read_csv('test.csv', encoding='cp950')

,股票代號╲年度,2022,2021,2020,2019,2018,2017,2016,2015,2014,...,1979,1978,1977,1976,1975,1974,1973,1972,1971,1970
0,50,4.09,2.48,3.71,3.61,3.55,3.05,1.28,3.01,2.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51,4.35,3.22,2.97,3.68,3.13,2.89,2.78,3.06,2.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,52,9.63,0.82,2.4,1.83,3.24,2.88,5.99,0,6.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,53,2.91,2.56,2.88,4.18,3.14,3.34,3.53,3.22,3.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55,3.7,3.48,3.68,3.88,3.75,3.23,3.42,3.01,2.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2089,9955,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2090,9957,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2091,9958,3.02,2.64,1.01,0.43,0.46,2.62,4,3.12,6.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2092,9960,6.57,3.53,2.81,1.46,0,1.23,1.86,3.82,5.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
